# Production mix - Julia/JuMP

## Situation
You own a boutique pottery business, making and selling two types of large ornamental products called Lunar Orb and Solar Disc. Given constraints on staff hours, available materials, and product sales, your objective is to maximize the total profit margin from the shop.

## Implementation
Linear Program (LP), using Julia/JuMP.

## Source
Replicates a Python model described in article "Production mix - Model 5, Pyomo using def" at https://www.solvermax.com/blog/production-mix-model-5-pyomo.

In [1]:
# Libraries

using JuMP
using Printf, DataFrames, PrettyTables
using JSON
using HiGHS, Cbc, GLPK

In [2]:
# Globals

filename = "productiondata.json"

struct ParsedData   # struct must be defined at global level
    name::String
    hours::Real
    kg::Real
    saleslimit::Real
    varinitial::Real
    varlbounds::Real
    varubounds::Real
    products::Vector
    people::Dict
    materials::Dict
    sales::Dict
    margin::Dict
    engine::String
    timelimit::Int64
end

In [3]:
# Data

function parsefiledata(filedata)   # Parse xml data file and return a struct containing all the data
    name = filedata["Name"]
    hours = filedata["Hours"]
    kg = filedata["kg"]
    saleslimit = filedata["SalesLimit"]
    varinitial = filedata["VarInitial"]
    varlbounds = filedata["VarLBounds"]
    varubounds = filedata["VarUBounds"]
    products = sort(collect(keys(filedata["Coefficients"])))
    people = Dict()
    materials = Dict()
    sales = Dict()
    margin = Dict()
    for p in products
        people[p] = filedata["Coefficients"][p]["People"]
        materials[p] = filedata["Coefficients"][p]["Materials"]
        sales[p] = filedata["Coefficients"][p]["Sales"]
        margin[p] = filedata["Coefficients"][p]["Margin"]
    end
    engine = filedata["Engine"]
    timelimit = filedata["TimeLimit"]
    data = ParsedData(name, hours, kg, saleslimit, varinitial, varlbounds, varubounds, products, people, materials, sales, margin, engine, timelimit)
    return data
end;

In [4]:
# Model

function definemodel(data)   # Define the model
    model = Model()
    @variable(model, data.varlbounds <= production[p in data.products] <= data.varubounds)
    @constraint(model, cpeople, sum(data.people[p] * production[p] for p in data.products) <= data.hours)
    @constraint(model, cmaterials, sum(data.materials[p] * production[p] for p in data.products) <= data.kg)
    @constraint(model, csales, sum(data.sales[p] * production[p] for p in data.products) <= data.saleslimit)
    @objective(model, Max, sum(data.margin[p] * production[p] for p in data.products))
    return model
end;

function modeloptions!(model, data)   # Set model options. Function name includes ! to follow convention for functions that alter their arguments
    unset_silent(model)   # Verbose output from the solver, c.f. set_silent()
    set_optimizer_attribute(model, "presolve", "on")   # Presolve
    set_optimizer_attribute(model, "time_limit", data.timelimit)  # Time limit, seconds
    if data.engine == "GLPK"
        set_optimizer(model, GLPK.Optimizer)
    elseif data.engine == "Cbc"
        set_optimizer(model, GLPK.Optimizer)
    elseif data.engine == "HiGHS"
        set_optimizer(model, HiGHS.Optimizer)
    else
        println( "Unknown solver. Note that names are case sensitive.")
    end
    return model
end;

In [5]:
# Reporting

function variable_report(xi, report)   # Extract variable characteristics. Source: https://jump.dev/JuMP.jl/stable/tutorials/linear/lp_sensitivity
    return (
        name = name(xi),
        lower_bound = has_lower_bound(xi) ? lower_bound(xi) : -Inf,
        value = value(xi),
        upper_bound = has_upper_bound(xi) ? upper_bound(xi) : Inf,
        reduced_cost = reduced_cost(xi),
        obj_coefficient = coefficient(objective_function(model), xi),
        allowed_decrease = report[xi][1],
        allowed_increase = report[xi][2],
    )
end;

function constraint_report(c::ConstraintRef, report)   # Extract constraint characteristics. Source: https://jump.dev/JuMP.jl/stable/tutorials/linear/lp_sensitivity
    return (
        name = name(c),
        value = value(c),
        rhs = normalized_rhs(c),
        slack = normalized_rhs(c) - value(c),
        shadow_price = shadow_price(c),
        allowed_decrease = report[c][1],
        allowed_increase = report[c][2],
    )
end;

function output(model, data)   #   Write output, depnding on solve status
    println("\nModel: $(data.name)")
    println("Solver: $(data.engine)")
    println("Termination status: $(termination_status(model))")
    duals = ConstraintRef[]
    if termination_status(model) == MOI.OPTIMAL
        report = lp_sensitivity_report(model)
        @printf "Total margin = \$%.2f\n" objective_value(model)
        variable_df = DataFrame(variable_report(xi, report) for xi in all_variables(model))
        constraint_df = DataFrame(constraint_report(ci, report) for (F, S) in list_of_constraint_types(model) for ci in all_constraints(model, F, S) if F == AffExpr)
        pretty_table(variable_df, nosubheader = true, formatters = ft_printf("%.4f"))
        pretty_table(constraint_df, nosubheader = true, formatters = ft_printf("%.4f"))    
    else
        println("No solution")
    end
    print(model)
end;

In [6]:
# Main

filedata = JSON.parsefile(filename);
data = parsefiledata(filedata)
model = definemodel(data)
model = modeloptions!(model, data)
optimize!(model)
output(model, data)

Running HiGHS 1.5.1 [date: 1970-01-01, git hash: 93f1876e4]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
3 rows, 2 cols, 6 nonzeros
3 rows, 2 cols, 6 nonzeros
Presolve : Reductions: rows 3(-0); columns 2(-0); elements 6(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          2     3.0769230769e+03 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  3.0769230769e+03
HiGHS run time      :          0.01

Model: Boutique pottery shop - Julia/JuMP
Solver: HiGHS
Termination status: OPTIMAL
Total margin = $3076.92
┌───────────────────┬─────────────┬─────────┬─────────────┬──────────────┬─────────────────┬──────────────────┬──────────────────┐
│              name │ lower_bound │   value │ upper_bound │ reduced_cost │ obj_coefficient │ allowed_decrease │ allowed_increase │
├───